**Importing libraries**

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Reading data**

In [ ]:
tt1 = pd.read_csv('AzureReadings_at_a_timestamp.csv')

In [ ]:
tt1.head()

In [ ]:
tt1['timestamp'] =  pd.to_datetime(tt1['timestamp'])

In [ ]:
tt1 = tt1.set_index('timestamp')

In [ ]:
tt1.head()

In [ ]:
tt = tt1

In [ ]:
dataset = tt.values
dataset = dataset.astype('float32')

In [ ]:
len(dataset)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
print(len(train), len(test))

In [ ]:
def create_training_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :3]
        dataX.append(a)
        dataY.append(dataset[i + look_back, :])
    return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 5
trainX, trainY = create_training_dataset(train, look_back=look_back)
testX, testY = create_training_dataset(test, look_back=look_back)

**Building Model**

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.GRU(128, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(keras.layers.Dense(3))

In [ ]:
adamOpt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
model.compile(loss='mean_squared_error', optimizer=adamOpt, metrics=['mae'])

In [ ]:
model.summary()

**Training Model**

In [ ]:
history = model.fit(trainX, trainY, validation_split=0.25,epochs=20, batch_size=64, verbose=2)

**Predicting**

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
trainY = scaler.inverse_transform(trainY)
trainPredict = scaler.inverse_transform(trainPredict)
testY = scaler.inverse_transform(testY)
testPredict = scaler.inverse_transform(testPredict)

**Evaluating Model**

1. Evaluation metrics: RMSE(Root Mean Square Error)

In [ ]:
trainScore = math.sqrt(mean_squared_error(trainY[:], trainPredict[:]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:], testPredict[:]))
print('Test Score: %.2f RMSE' % (testScore))

2. Evaluation metrics: MAE(Mean Absolute Error)

In [ ]:
from sklearn.metrics import mean_absolute_error
trainScore = (mean_absolute_error(trainY[:], trainPredict[:]))
print('Train Score: %f MAE' % (trainScore))
testScore = math.sqrt(mean_absolute_error(testY[:], testPredict[:]))
print('Test Score: %f MAE' % (testScore))

3. Evaluation metric: MAPE(Mean absolute percentage error)

**Percentage errors**

The percentage error is given by  
p
t
=
100
e
t
/
y
t
 . 

Percentage errors have the advantage of being unit-free, and 
so are frequently used to compare forecast performances between data sets. 

The most commonly used measure is:
Mean absolute percentage error: MAPE
=
mean
(
|
p
t
|
)


In [ ]:
trainScore2 = np.mean(np.abs(trainPredict - trainY)/np.abs(trainY))
print('Train Score: %f MAPE' % (trainScore2))
testScore2 = np.mean(np.abs(testPredict - testY)/np.abs(testY))
print('Test Score: %f MAPE' % (testScore2))

4. Evaluation metric: Correaltion

In [ ]:
trainScore3 = np.corrcoef(trainPredict, trainY)[0,1]
print('Train Score: %f COR' % (trainScore3))
testScore2 = np.corrcoef(testPredict, testY)[0,1]
print('Test Score: %f COR' % (testScore2))

**Plotting Graphs**

In [ ]:
index=tt.index
TestY= pd.DataFrame(testY,columns=['min_cpu','max_cpu','avg_cpu'])
PredY=pd.DataFrame(testPredict,columns=['min_cpu','max_cpu','avg_cpu'])

x=index[-1722:]
fig, axs = plt.subplots(3,figsize=(10,15))

axs[0].plot(x,TestY.min_cpu,'.',label='Test min CPU',color='red')
axs[0].plot(x,PredY.min_cpu,'--',label='Predicted min CPU',color='black')
axs[0].legend()
axs[1].plot(x,TestY.max_cpu,'.',label='Test max CPU',color='magenta')
axs[1].plot(x,PredY.max_cpu,'--',label='Predicted max CPU',color='navy')
axs[1].legend()
axs[2].plot(x,TestY.avg_cpu,'.',label='Test avg CPU',color='orange')
axs[2].plot(x,PredY.avg_cpu,'--',label='Predicted avg CPU',color='darkgreen')
axs[2].legend()
for ax in axs.flat:
    ax.set(xlabel='Timestamp', ylabel='Workload (CPU)',autoscale_on=True)
for ax in axs:
    ax.label_outer()
#fig.suptitle('Prediction of Workload on Azure cloud at a particular timestamp',fontsize=20)
plt.savefig('/content/drive/My Drive/azure download/data/LSTM output.png', dpi = 300)
plt.show()